In [ ]:
import magmaforge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
moon_press = pd.read_csv('data/Moon_press.csv',header=0)

In [ ]:
def transform_mass(m):
    return m**(2/3)

def apply_constrained_poly_endpoints(poly_coef:np.array, Pmax:float):
    xends = np.array([0, 1])
    yends = np.polyval(poly_coef, [0,1])-(1-xends)*Pmax

    poly_coef = poly_coef.copy()
    poly_coef[-1] -= yends[0]
    poly_coef[-2] -= yends[1]-yends[0]
    return poly_coef

def fit_press_poly(mass_frac:np.ndarray, P:np.ndarray, deg:int=4):
    mval = transform_mass(mass_frac)

    poly_coef0 = np.polyfit(mval, P, deg)
    Pmax = P.max()
    poly_coef = apply_constrained_poly_endpoints(poly_coef0, Pmax)
    return poly_coef

def eval_press(mass_frac:np.ndarray, poly_coef:np.ndarray):
    if type(mass_frac) is not np.ndarray:
        mass_frac = np.array(mass_frac)
        
    mval = transform_mass(mass_frac)
    return np.polyval(poly_coef, mval)
    


m_frac = np.linspace(0, 1,101)

poly_coef = fit_press_poly(moon_press['mass_frac'], moon_press['P'])


mval = transform_mass(moon_press['mass_frac'])
mmod = transform_mass(m_frac)

# poly_coef = np.polyfit(mval, moon_press['P'],4)
Pmax = moon_press['P'].max()
# poly_coef = apply_constrained_poly_endpoints(poly_coef, Pmax)




plt.figure()
plt.plot(mval, moon_press['P']-(1-mval)*Pmax, '.-')
plt.plot(mmod, np.polyval(poly_coef, mmod)-(1-mmod)*Pmax, 'r--')



In [ ]:
plt.figure()
plt.plot(moon_press['mass_frac'], moon_press['P'], 'o')
plt.plot(m_frac, eval_press(m_frac, poly_coef), 'r--')
plt.plot(m_frac, (1-m_frac)*Pmax, ':', color=[.5,.5,.5])

mass_samples = 1-.9**np.arange(15)
plt.plot(mass_samples, eval_press(mass_samples, poly_coef), 'mx')

In [ ]:
comp={
    'BSE': {
        'SiO2': 45.97,
        'TiO2':  0.18,
        'Al2O3': 4.77,
        'Fe2O3': 1e-7,
        'FeO':   8.24,
        'MnO':   0.0,
        'MgO':  36.66,
        'CaO':   3.78,
        'Na2O':  0.35,
        'K2O':   0.04,},
    'BSE_MS95': {
        'SiO2': 45.5,
        'TiO2':  0.00001,
        'Al2O3': 4.50,
        'FeO':   8.15,
        'MnO':   0.0,
        'MgO':  38.3,
        'CaO':   3.58,
        'Na2O':  0.00001,
        'K2O':   0.0,},
}


P = 24e3
T0 = 2200.0
Tfinal = 1600 + 273-100

BSE = comp['BSE_MS95']
dNNO = -4

model_name = 'pMELTS'

In [ ]:
comp_IW0 = magmaforge.System.adjust_init_redox(T0, P, BSE, dNNO, 'NNO', model_name)
Tliq = magmaforge.System.get_liquidus_temp(T0, P, comp_IW0, model_name)
comp_IW = magmaforge.System.adjust_init_redox(Tliq, P, BSE, dNNO, 'NNO', model_name)

print(Tliq)

In [ ]:
def get_liquidus_entropy(comp, Tliq, P, model_name):
    sys_T = magmaforge.System(comp=comp_IW, T0=Tliq, P=P,
                            melt_frac_cutoff=.15, model_name=model_name)

    return sys_T.total_entropy

In [ ]:
Sliq = get_liquidus_entropy(comp_IW, Tliq, P, model_name)
print(Sliq)

sys_S = magmaforge.System(comp=comp_IW, T0=Tliq, P=P, frac_remains=1e-5,
                          min_potential='H', S0=Sliq, model_name=model_name)

print(sys_S.total_entropy)
print(sys_S.state.conditions)
print(f"Fe2O3 = {sys_S.state.liquid.comp['Fe2O3']}")

In [ ]:
Tfinal

In [ ]:
thresh_xtal_frac = 0.1
dS = 0.5

In [ ]:
def crystalize_to_threshold(sys_S, P, thresh_xtal_frac, dS=1.0, Tfinal=1779.0):
    while(sys_S.melt_fraction > 1-thresh_xtal_frac):
        sys_S.cool(P=P, dS=dS)
        print(f'   T = {sys_S.T}')
        if sys_S.T < Tfinal:
            break

In [ ]:
while(sys_S.mass_fraction > 0.01):
    print(f'dS = {dS}')
    crystallized_frac = 1 - sys_S.mass_fraction
    P_GPA = eval_press(crystallized_frac, poly_coef)
    P = P_GPA*1e4  # convert to bars
    print(f'P = {P}')
    crystalize_to_threshold(sys_S, P, thresh_xtal_frac, dS=dS)
    sys_S.fractionate_phases()
    # sys_S.cool(P=P, dS=0.01*dS)
    # dS = np.maximum(dS*(1-thresh_xtal_frac),0.25)
    

In [ ]:
1-sys_S.mass_fraction

In [ ]:
plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_melt_frac(),'.-', label='S')
plt.xlabel('T')
plt.ylabel('melt frac')
plt.legend()

plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_total_mass(),'.-', label='S')
plt.xlabel('T')
plt.ylabel('mass frac')
plt.legend()

plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_total_entropy(),'.-', label='S')
plt.xlabel('T')
plt.ylabel('Stot')

In [ ]:
sys_S.history.phase_frac_table

In [ ]:
phase_frac_tbl = sys_S.history.phase_frac_table
phase_frac_tbl.drop

In [ ]:
phs_present_tbl = phase_frac_tbl.loc[:,(phase_frac_tbl != 0).any(axis=0)]


In [ ]:
crystallize_tbl = phs_present_tbl.drop(columns='Liquid')
mineral_assem_tbl = (crystallize_tbl.T/crystallize_tbl.sum(axis=1)).T

mineral_assem_tbl.index = 100-sys_S.history.get_total_mass()

In [ ]:
mineral_assem_tbl.plot.area(lw=0)
plt.xlabel('Solidification of LMO [%]')
plt.ylabel('Crystallized Mineral Fraction')

In [ ]:
mineral_assem_tbl

In [ ]:
magmaforge.plot.magma_evolution(sys_S.history)

In [ ]:
1234+273
